In [72]:
import pandas as pd
import numpy as np

In [73]:
path = '/Users/laszlo.sandor/Documents/github/106/adat/part/google_trends/gtrend1118/'

In [74]:
megyelist = ['bacskiskun','baranya','baz','bekes','budapest','csongrad','fejer','gyms','hb','heves','jnsz','ke','nograd','pest','somogy','szszb','tolna','vas','veszprem','zala']
megyedict = {'bacskiskun':3,'baranya':2,'baz':5,'bekes':4,'budapest':1,'csongrad':6,'fejer':7,'gyms':8,'hb':9,'heves':10,'jnsz':11,'ke':12,'nograd':13,'pest':14,'somogy':15,'szszb':16,'tolna':17,'vas':18,'veszprem':19,'zala':20}

In [75]:
top5_megyek = pd.DataFrame()
top5list = []
for m in megyelist:
    filename = 'top5_' + m + '.csv'
    fullpath = path + filename
    df = pd.read_csv(fullpath,engine='c', dtype={'Fidesz':np.uint8,'Jobbik':np.uint8,'Hungarian Socialist Party':np.uint8,'Democratic Coalition':np.uint8,'Politics Can Be Different':np.uint8})#,parse_dates=[0])
    df['megye'] = megyedict[m]
    top5list.append(df)
top5_megyek = pd.concat(top5list)

In [76]:
top5_megyek.head()

,Month,Fidesz,Jobbik,Hungarian Socialist Party,Democratic Coalition,Politics Can Be Different,megye
0,2011-01,21,17,13,4,4,3
1,2011-02,23,14,14,0,0,3
2,2011-03,21,17,8,4,8,3
3,2011-04,32,37,18,5,0,3
4,2011-05,48,22,9,0,4,3


In [77]:
small3_megyek = pd.DataFrame()
small3list = []
for m in megyelist:
    filename = 'small3_' + m + '.csv'
    fullpath = path + filename
    df = pd.read_csv(fullpath,engine='c', dtype={'Fidesz':np.uint8, 'Jobbik':np.uint8, 'Together':np.uint8, 'Momentum Movement':np.uint8, 'Dialogue for Hungary':np.uint8})
    df['megye'] = megyedict[m]
    small3list.append(df)
small3_megyek = pd.concat(small3list)

In [78]:
top5_megyek.rename(columns={'Fidesz':'fidesz','Jobbik':'jobbik','Hungarian Socialist Party':'mszp','Democratic Coalition':'dk','Politics Can Be Different':'lmp'},inplace=True)
small3_megyek.rename(columns={'Fidesz':'fidesz','Jobbik':'jobbik','Together':'egyutt', 'Momentum Movement':'momentum', 'Dialogue for Hungary':'pm'},inplace=True)

In [79]:
top5melt = pd.melt(top5_megyek, id_vars=['Month','megye'], value_vars=['fidesz', 'jobbik', 'mszp', 'dk', 'lmp'])
top5melt['value'].astype(int)
top5melt.rename(columns = {'variable':'party','value':'search_intensity'},inplace=True)
top5meltpivot=top5melt.pivot_table(values='search_intensity', index=['Month', 'party'], columns='megye').reset_index()

In [80]:
small3melt = pd.melt(small3_megyek.drop(['fidesz', 'jobbik'],axis=1), id_vars=['Month','megye'], value_vars=[  'momentum', 'egyutt', 'pm'])
small3melt['value'].astype(int)
small3melt.rename(columns = {'variable':'party','value':'search_intensity'},inplace=True)
small3meltpivot=small3melt.pivot_table(values='search_intensity', index=['Month', 'party'], columns='megye').reset_index()

In [81]:
all_megye_melt = top5melt.append(small3melt)

In [82]:
all_megye_melt.head()

,Month,megye,party,search_intensity
0,2011-01,3,fidesz,21
1,2011-02,3,fidesz,23
2,2011-03,3,fidesz,21
3,2011-04,3,fidesz,32
4,2011-05,3,fidesz,48


In [83]:
# setting all searches for Momentum to 0 before 2017, no need to do the same for other new parties as they existed for the 2014 elections
all_megye_melt.loc[(all_megye_melt.Month < '2017') & (all_megye_melt.party == 'momentum'),'search_intensity'] = 0

In [84]:
amm_gbsum = all_megye_melt.groupby(['Month','megye','party'])['search_intensity'].sum()
minden_megye_aranyok = amm_gbsum.groupby(level=[0,1]).apply(lambda x: x / float(x.sum())).reset_index()

In [85]:
minden_megye_aranyok.head()

,Month,megye,party,search_intensity
0,2011-01,1,dk,0.089109
1,2011-01,1,egyutt,0.029703
2,2011-01,1,fidesz,0.376238
3,2011-01,1,jobbik,0.247525
4,2011-01,1,lmp,0.049505


In [86]:
minden_megye_aranyok.to_csv('GTrend_megye_aranyok1118.csv',index=False)